In [14]:
import os
import re
import sys
import hashlib
import socket
import requests
import json
import time
from time import time
from urllib.parse import urlparse
from flask import request
from operator import itemgetter
from datetime import datetime, timezone
from pyp2p.net import *

from config import Config

## Test Blockchain Endpoints

In [82]:
host = 'http://0.0.0.0:5000'

In [116]:
nodes = requests.get(host+"/nodes/getNodes").content
nodes

b'{"nodes": ["0.0.0.0:5002"]}'

In [125]:
host1 = 'http://0.0.0.0:5001'
nodes1 = requests.get(host1+"/nodes/getNodes").content
nodes1

b'{"nodes": ["0.0.0.0:5000"]}'

In [170]:
# wallet = requests.post(host+'/wallet/new', json={'password': 'lalalend'}).content
# wallet

In [256]:
regNode = requests.post(host+"/nodes/register", json={"nodes": ['http://0.0.0.0:5001']}).content
regNode

b'{"message":"New nodes have been added","total_nodes":["0.0.0.0:5002","0.0.0.0:5001"]}\n'

In [257]:
login = requests.post(host+'/wallet/login', json={'password': 'lalalend'}).content
login

b'{"ADDRESS":"z01x00a61643807d1f0bfe6dbb609311de768fb41430479b6564ca5f2c4f06","MSG":true}\n'

In [69]:
# logout = requests.get(host+'/wallet/logout').content
# logout

b'{"MSG":true}\n'

In [258]:
tradeTx = {
    'type': 'trade',
    'sender': json.loads(wallet)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 30000.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 60000.0,
    'comissionAmount':2
}

txReq = requests.post(host+'/transactions/new', json=tradeTx).content
txReq

b'{"MSG":2,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [259]:
tradeTx = {
    'type': 'trade',
    'sender': json.loads(wallet)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 30000.0,
    'send': 'usdt',
    'sendVol': 60000.0,
    'get': 'btc',
    'getVol': 2.0,
    'comissionAmount':2
}

txReq = requests.post(host+'/transactions/new', json=tradeTx).content
txReq

b'{"MSG":2,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [261]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":2,"message":"New Block Forged","pool_syncing":"Tx pool synced among 1 nodes","previous_hash":"9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393","proof":22976,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"d32902ea13e44d3a9f905d99620d0892","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723151.043912,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":30000.0,"recieveAmount":60000.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","sendAmount":2.0,"sender":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","symbol":"btc/usdt","timestamp":1629723151.057158,"tradeTxId":"8e85b87e9298b3d39259884f5edc56eb516ec865d96333783cc2c4e6cc53f22e"},{"comissionAmount":2.0,"contract":"usdt","price":30000.0,"recieveAmount":2.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b370559

In [269]:
ordersPool = requests.get(host+'/getTradeOrders').content
ordersPool

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":6.0,"price":3.0,"send":"btc","sendVol":2.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723312.347348,"tradeTxId":"fb7456550faa82f198c6026e488ce59602ae9a013ad08fa808e2b6bdff76e856"},{"comissionAmount":2.0,"get":"btc","getVol":2.0,"price":3.0,"send":"usdt","sendVol":6.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723314.20756,"tradeTxId":"43fd5abe90609c230698a44803fd474eb96d75e401a35133f039d6976fc34555"}]}\n'

In [270]:
txPool = requests.get(host+'/getTxPool').content
txPool

b'{"txPool":[]}\n'

In [274]:
chain = requests.get(host+'/chain').content
chain

b'{"chain":[{"index":1,"previousHash":"9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681","proof":42,"timestamp":1629032178.368946,"transactions":["ascl,sdc,d[v,[d,vsdcaaxasx]]","lv,sldc,sdc ;,d;d"]},{"index":2,"previousHash":"9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393","proof":22976,"timestamp":1629723151.057186,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"d32902ea13e44d3a9f905d99620d0892","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723151.043912,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":30000.0,"recieveAmount":60000.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","sendAmount":2.0,"sender":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","symbol":"btc/usdt","timestamp":1629723151.057158,"tradeTxId":"8e85b87e9298b3d39259884f5edc56eb516ec865d96333783c

### NODE 1

In [265]:
host1 = 'http://0.0.0.0:5001'

In [74]:
wallet1 = requests.post(host1+'/wallet/new', json={'password': 'lalalend'}).content
wallet1

b'{"ADDRESS":"z01x001ddb88cc07c6d9179c51b370128d07109a650cba29c5889c078e6bb0"}\n'

In [266]:
login1 = requests.post(host1+'/wallet/login', json={'password': 'lalalend'}).content
login1

b'{"ADDRESS":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","MSG":true}\n'

In [267]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 6.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [268]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'usdt',
    'sendVol': 6.0,
    'get': 'btc',
    'getVol': 2.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [271]:
ordersPool1 = requests.get(host1+'/getTradeOrders').content
ordersPool1

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":6.0,"price":3.0,"send":"btc","sendVol":2.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723312.347348,"tradeTxId":"fb7456550faa82f198c6026e488ce59602ae9a013ad08fa808e2b6bdff76e856"},{"comissionAmount":2.0,"get":"btc","getVol":2.0,"price":3.0,"send":"usdt","sendVol":6.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723314.20756,"tradeTxId":"43fd5abe90609c230698a44803fd474eb96d75e401a35133f039d6976fc34555"}]}\n'

In [272]:
mineReq1 = requests.get(host1+'/mine').content
mineReq1

b'{"index":3,"message":"New Block Forged","pool_syncing":"Tx pool synced among 1 nodes","previous_hash":"49c4fa4aada5dec35ce6f03f9de1465628b9513738ce8d3dd1f18cb17d0f2a8f","proof":119240,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"895cc4a0852d41a9a9c4f7f0ad55d296","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723377.434202,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":3.0,"recieveAmount":6.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAmount":2.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723377.440518,"tradeTxId":"fb7456550faa82f198c6026e488ce59602ae9a013ad08fa808e2b6bdff76e856"},{"comissionAmount":2.0,"contract":"usdt","price":3.0,"recieveAmount":2.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAm

In [177]:
txPool1 = requests.get(host1+'/getTxPool').content
txPool1

b'{"txPool":[]}\n'

In [273]:
chain1 = requests.get(host1+'/chain').content
chain1

b'{"chain":[{"index":1,"previousHash":"9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681","proof":42,"timestamp":1629032178.368946,"transactions":["ascl,sdc,d[v,[d,vsdcaaxasx]]","lv,sldc,sdc ;,d;d"]},{"index":2,"previousHash":"9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393","proof":22976,"timestamp":1629723151.057186,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"d32902ea13e44d3a9f905d99620d0892","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723151.043912,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":30000.0,"recieveAmount":60000.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","sendAmount":2.0,"sender":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","symbol":"btc/usdt","timestamp":1629723151.057158,"tradeTxId":"8e85b87e9298b3d39259884f5edc56eb516ec865d96333783c